# 5DEEP - Projet

## Imports

In [21]:
# OS and filesystem
from pathlib import Path

# Data
import pandas

# Console output
from colorama import Fore, Style
from tqdm.notebook import tqdm

## Constants

In [22]:
PARENT_FOLDER = Path.cwd()
DATA_FOLDER = PARENT_FOLDER / ".." / "data"
URBAN_DATASET_FOLDER = DATA_FOLDER / "UrbanSound8K"

## Dataset initialization
Load the dataset and check if it's fully present.

In [23]:
data = pandas.read_csv(URBAN_DATASET_FOLDER / "metadata" / "UrbanSound8K.csv")
data.head(n=5)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [24]:
missing_files_count = 0

for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Checking the dataset integrity..."):
    file_path = URBAN_DATASET_FOLDER / "audio" / f"fold{row['fold']}" / row["slice_file_name"]

    if not file_path.is_file():
        missing_files_count += 1
        print(f"{Style.DIM}{Fore.WHITE}Missing file at \"{file_path}\"{Style.RESET_ALL}")

if missing_files_count == 0:
    print(f"{Fore.GREEN}Successful verification of the dataset. No missing files were detected.{Style.RESET_ALL}")
else:
    print(f"{Fore.YELLOW}Successful verification of the dataset. {missing_files_count} file{'s are ' if missing_files_count > 1 else ' is '} missing. You should consider re-downloading the dataset.{Style.RESET_ALL}")

Checking the dataset integrity...:   0%|          | 0/8732 [00:00<?, ?it/s]

Successful verification of the dataset. No missing files were detected.
